In [6]:
import pandas as pd

In [7]:
file = "75YearsofEngineering"

### Join tables to get Figure-Caption pairs

In [8]:
links = pd.read_pickle(f'results/{file}_caps.pkl')
links.head(3)

,figure_id,figure_caption_id
20,75YearsofEngineering_15_0,75YearsofEngineering_15_3
23,75YearsofEngineering_15_1,75YearsofEngineering_15_2
29,75YearsofEngineering_16_0,75YearsofEngineering_16_6


In [9]:
ocr_results = pd.read_pickle(f'results/{file}_ocr.pkl')
ocr_results.head(3)

,id,page,xyxy,image_array,type,type_str,type_conf,ocr_text
0,75YearsofEngineering_0_0,0,"[[24.6069278717041, 190.01702880859375, 547.62...","[[[180, 133, 111], [180, 133, 111], [182, 135,...",3,figure,0.828341,NaN
1,75YearsofEngineering_0_1,0,"[[245.99098205566406, 20.62215805053711, 324.8...","[[[178, 129, 111], [177, 127, 109], [178, 128,...",2,abandon,0.756952,
2,75YearsofEngineering_0_2,0,"[[77.55631256103516, 125.02350616455078, 501.7...","[[[180, 130, 112], [181, 130, 113], [179, 129,...",0,title,0.474390,75 Years of Engineering


In [10]:
captions = pd.merge(links, ocr_results, left_on='figure_caption_id', right_on='id')[["figure_id", "ocr_text"]]
captions = captions.rename(columns={'ocr_text': 'caption'})
captions.head(6)

,figure_id,caption
0,75YearsofEngineering_15_0,The ceremony was attended by over three hundr...
1,75YearsofEngineering_15_1,"From left to right: The Hon. Allen Lee,Dr.R.C..."
2,75YearsofEngineering_16_0,The Hon H.K. Cheng and Dr. Wang Gungwu enjoyi...
3,75YearsofEngineering_16_1,Y.C cocktail hou
4,75YearsofEngineering_16_2,"Former professor, Mr. W.A. Reynolds and Mrs R..."
5,75YearsofEngineering_16_3,"Prof. W.S. Leung, serving the longest- teache..."


In [11]:
figures_data = pd.merge(captions, ocr_results, left_on='figure_id', right_on='id')[["figure_id", "caption","image_array", "page"]]
figures_data.head(6)

KeyboardInterrupt: 

Converts the BGR array image into base64 image

In [7]:
from PIL import Image
import io
import base64

def bgr2base64(arr):
    arr = arr[:, :, ::-1]
    im = Image.fromarray(arr.astype("uint8"))
    rawBytes = io.BytesIO()
    im.save(rawBytes, "JPEG")
    rawBytes.seek(0)  # return to the start of the file
    return base64.b64encode(rawBytes.read())

figures_data["image_base64"] = figures_data["image_array"].apply(bgr2base64)
figures_data.head(6)

,figure_id,caption,image_array,page,image_base64
0,75YearsofEngineering_15_0,The ceremony was attended by over three hundr...,"[[[252, 255, 253], [252, 255, 253], [252, 255,...",15,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
1,75YearsofEngineering_15_1,"From left to right: The Hon. Allen Lee,Dr.R.C...","[[[252, 255, 253], [252, 255, 253], [252, 255,...",15,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
2,75YearsofEngineering_16_0,The Hon H.K. Cheng and Dr. Wang Gungwu enjoyi...,"[[[249, 254, 252], [197, 217, 222], [126, 133,...",16,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
3,75YearsofEngineering_16_1,Y.C cocktail hou,"[[[251, 255, 253], [235, 244, 245], [229, 230,...",16,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
4,75YearsofEngineering_16_2,"Former professor, Mr. W.A. Reynolds and Mrs R...","[[[252, 255, 253], [243, 249, 248], [206, 210,...",16,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
5,75YearsofEngineering_16_3,"Prof. W.S. Leung, serving the longest- teache...","[[[252, 255, 253], [252, 255, 253], [253, 254,...",16,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...


In [8]:
figures_data.drop(["image_array"], inplace=True, axis=1)
figures_data.rename(columns={"figure_id": "id"}, inplace=True)
figures_data.head(3)

,id,caption,page,image_base64
0,75YearsofEngineering_15_0,The ceremony was attended by over three hundr...,15,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
1,75YearsofEngineering_15_1,"From left to right: The Hon. Allen Lee,Dr.R.C...",15,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
2,75YearsofEngineering_16_0,The Hon H.K. Cheng and Dr. Wang Gungwu enjoyi...,16,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...


### Filter out plain text

In [15]:
text_data = ocr_results[ocr_results["type_str"].isin(["plain text", "table"])][["id", "page", "ocr_text"]]
text_data.head(10)

,id,page,ocr_text
3,75YearsofEngineering_0_3,0,75 Years of Engineering
7,75YearsofEngineering_2_2,2,Faculty of Engineering University of Hong Kong
11,75YearsofEngineering_2_6,2,75 Years of Engineering
13,75YearsofEngineering_3_1,3,Faculty of Engineering University of Hong Kon...
14,75YearsofEngineering_4_0,4,The Hon. Allen Lee Dr. R.C.T. Ho Professor Y....
15,75YearsofEngineering_4_1,4,Days in the Department of Mechanical Engineer...
16,75YearsofEngineering_4_2,4,"His Excellency Sir David Wilson, KCMG Sir Sze..."
17,75YearsofEngineering_4_3,4,Reminiscences of the Faculty of Engineering 8...
18,75YearsofEngineering_4_4,4,Reminiscences of My Early Days with the Facul...
19,75YearsofEngineering_4_5,4,The Don in Action Again 72 (E.A.Bruges)


### Create Chroma DB

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="BAAI/bge-m3"
)

client = chromadb.PersistentClient()

/home/cxiang/aiha_new/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_collection = client.get_or_create_collection(name="text_data", embedding_function=sentence_transformer_ef,)
text_collection.add(
    documents=text_data["ocr_text"].tolist(),
    metadatas=text_data[["page"]].to_dict("records"),
    ids=text_data["id"].tolist()
)

NameError: name 'client' is not defined

In [5]:
text_collection = client.get_or_create_collection(name="text_data", embedding_function=sentence_transformer_ef,)

text_collection.query(
    query_texts=["president of HKU"],
    n_results=10
)

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None,
 'metadatas': [[]],
 'distances': [[]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}